In [9]:
pip install "snowflake-connector-python[pandas]"


     |████████████████████████████████| 24.6MB 7.3MB/s eta 0:00:01
     |████████████████████████████████| 450kB 58.3MB/s eta 0:00:01
     |████████████████████████████████| 71kB 3.4MB/s  eta 0:00:01
     |████████████████████████████████| 61kB 5.7MB/s  eta 0:00:01
     |████████████████████████████████| 163kB 105.5MB/s eta 0:00:01
     |████████████████████████████████| 51kB 4.0MB/s  eta 0:00:01
     |████████████████████████████████| 143kB 106.1MB/s eta 0:00:01
     |████████████████████████████████| 204kB 91.0MB/s eta 0:00:01
     |████████████████████████████████| 4.4MB 91.8MB/s eta 0:00:01
     |████████████████████████████████| 194kB 104.2MB/s eta 0:00:01
     |████████████████████████████████| 2.1MB 93.0MB/s eta 0:00:01
     |████████████████████████████████| 71kB 4.3MB/s s eta 0:00:01
     |████████████████████████████████| 512kB 90.8MB/s eta 0:00:01
     |████████████████████████████████| 112kB 103.1MB/s eta 0:00:01
     |████████████████████████████████| 12.4MB 101.5MB/s eta 

Note: you may need to restart the kernel to use updated packages.


In [1]:
from snowflake.snowpark import Session
import configparser
def get_session():
    parser = configparser.ConfigParser()
    # Add the credential file name here
    parser.read('config.ini')

    connection_params = dict(user=parser['Credentials']['user'], 
                         password=parser['Credentials']['password'], 
                         account=parser['Credentials']['account'], 
                         warehouse=parser['Credentials']['warehouse'], 
                         database=parser['Credentials']['database'],
                         schema=parser['Credentials']['schema'], 
                         role=parser['Credentials']['role'])

    session = Session.builder.configs(connection_params).create()
    return session

/tmp/pip_packages/snowflake/connector/options.py:103: UserWarning: You have an incompatible version of 'pyarrow' installed (10.0.1), please install a version that adheres to: 'pyarrow<8.1.0,>=8.0.0; extra == "pandas"'
  warn_incompatible_dep(


In [2]:
session = get_session()

#### Load the prepared data from snowflake

In [3]:
df = session.sql("SELECT * from {table}".format(table="predictive_maintenance_final"))

In [4]:
df.columns

['FAILURE',
 'METRIC5',
 'METRIC6',
 'DAY',
 'DAY_WEEK',
 'SECTOR',
 'OP_PERIOD',
 'DEV_RECONNECTED',
 'MNW1',
 'DIF_M6',
 'DIF_M5',
 'LOG_M2',
 'LOG_M3',
 'LOG_M4',
 'LOG_M7',
 'LOG_M9']

Prepare dataset for training

In [ ]:
df_final = df.to_pandas()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import recall_score, f1_score, roc_auc_score, confusion_matrix,classification_report

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import SGDClassifier

# from imblearn.over_sampling import SMOTE
# Tunning hiperparameters
from sklearn.pipeline import Pipeline

In [ ]:
X = df_final.drop(['FAILURE'], axis = 1)
y = df_final['FAILURE']
X_train, X_test, Y_train, Y_test = train_test_split(X,y,random_state = 0, test_size=0.25, shuffle = True)
X_train.reset_index(inplace = True, drop = True)
Y_train.reset_index(inplace = True, drop = True)

X_test.reset_index(inplace = True, drop = True)
Y_test.reset_index(inplace = True, drop = True)

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, random_state = 0, test_size=0.25)

In [ ]:
pipeline = Pipeline([
    ('OHE', OneHotEncoder(handle_unknown = 'ignore',sparse=False)),
#     ('SCALER', StandardScaler()),
    ('SGD', AdaBoostClassifier(random_state=0))
     ])

# low_card =['sector']
# sm = SMOTE(random_state=0)

# X_res, Y_res = preprocess(x_train, y_train,OHE,low_card,scaler,1,sm)   
# X_val, Y_val = preprocess(x_val, y_val,OHE,low_card,scaler,0,sm)

pipeline.fit(x_train,y_train)
y_pred = pipeline.predict(x_val) 

cf_matrix = confusion_matrix(y_val, y_pred)
# s = sns.heatmap(cf_matrix, annot=True).set(xlabel='Predicted label', ylabel='True label')

In [ ]:
cf_matrix

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_val, y_pred))